In [13]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MultiLabelBinarizer

In [14]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
pd.options.mode.chained_assignment = (
    None  # default='warn', this removes warning on dropping columns
)

In [15]:
df_16 = pd.read_stata("../Data/raw/2016/2016-ASR_Public_Use_File.dta")
df_17 = pd.read_stata("../Data/raw/2017/2017 ASR_Public_Use_File.dta")
df_18 = pd.read_stata("../Data/raw/2018/2018 ASR_Public_Use_File.dta")
df_19 = pd.read_stata("../Data/raw/2019/2019 ASR_Public_Use_File.dta")

df_16.name = "df_16"
df_17.name = "df_17"
df_18.name = "df_18"
df_19.name = "df_19"

df_16["survey_year"] = 2016
df_17["survey_year"] = 2017
df_18["survey_year"] = 2018
df_19["survey_year"] = 2019

C:\Users\Patrick\AppData\Local\Temp\ipykernel_14852\4165299392.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_16["survey_year"] = 2016
C:\Users\Patrick\AppData\Local\Temp\ipykernel_14852\4165299392.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_18["survey_year"] = 2018
C:\Users\Patrick\AppData\Local\Temp\ipykernel_14852\4165299392.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns 

In [16]:
col_16_ww = df_16.columns.values
col_17_ww = df_17.columns.values
col_18_ww = df_18.columns.values
col_19_ww = df_19.columns.values
all_cols_ww = np.union1d(
    np.union1d(col_16_ww, col_17_ww), np.union1d(col_18_ww, col_19_ww)
)

To begin, we are going to reduce our sample set to just 1 respondent per house. In the following cell we conduct this filtering, and remove person level weighting along with redundant household level weighting. This is in accordance with the example from pages 33-37 from the 2019 ASR User Guide_no_appendices PDF, which states the following: 

"For household-level analysis, you need to filter the data file so that you have one observation per household. The easiest way to do this is to select only observations where the value of the “respondent” variable is equal to 1.

After selecting the 1,506 observations where the respondent variable equals 1, you would use the weight variable “Weight_household” or the weight variable “Weight_household_pop” to get household-level estimates. These two household-level weight variables will produce the same estimates. However, when using the “Weight_household” variable the frequency counts will sum to the ASR sample size of 1,506 and when using the “Weight_household_pop” variable the frequency counts will sum to the population of 118,403.

The data file also includes 23 replicate weights for each of the four survey weights on the data file (“Weight_person,” “Weight_person_pop,” “Weight_household,”
34
“Weight_household_pop”). Replicate weights were created for each replicate sample to make it easier to estimate standard errors and confidence intervals which is covered in the section 5 of this user’s guide."

In [17]:
# Filter out weight columns, and remove the two we want to keep
weight_col = [col for col in all_cols_ww if col.startswith("Weight_")]
weight_col.remove("Weight_household")
weight_col.remove("Weight_household_pop")

for df_yr in [df_16, df_17, df_18, df_19]:
    for col_w in weight_col:
        if col_w in df_yr.columns.values:
            df_yr.drop(col_w, axis=1, inplace=True)

In [18]:
col_16 = df_16.columns.values
col_17 = df_17.columns.values
col_18 = df_18.columns.values
col_19 = df_19.columns.values
all_cols = np.union1d(np.union1d(col_16, col_17), np.union1d(col_18, col_19))

In [19]:
# Check that each dataframe has all of the same columns
col_df = pd.DataFrame(columns=["df_16", "df_17", "df_18", "df_19"], index=all_cols)

for df_yr in [df_16, df_17, df_18, df_19]:
    for col in all_cols:
        if col in df_yr.columns.values:
            col_df.loc[col, df_yr.name] = 1

col_df

,df_16,df_17,df_18,df_19
Weight_household,1,1,1,1
Weight_household_pop,1,1,1,1
cohort,1,1,1,1
hhid,1,1,1,1
numppl,1,1,1,1
personid,1,1,1,1
qn10a,1,1,1,1
qn10b,1,1,1,1
qn11a,1,1,1,1
qn11aa,1,1,1,1


In [34]:
df_all_yr = pd.concat([df_16, df_17, df_18, df_19])
df_all_yr.apply(lambda x: x.astype(str).str.lower())


# Filter data to only be 1 respondent per house
rsp_df = df_all_yr[df_all_yr["respondent"] == "respondent"]

print(len(rsp_df))

rsp_df.sample(n=25, random_state=42)

4535


,hhid,qn1a,numppl,qn1b,qn1c,qn1d,qn1f,qn1g,qn1h,qn1i,qn1jyear,qn1k,qn1l,qn2a,qn2b,qn3a,qn3b,qn4a,qn4b,qn4c,qn4e,qn4j,qn5a,qn5b,qn5c,qn6a,qn6b,qn7,qn8a,qn8b,qn9,qn10a,qn10b,qn11a,qn11aa,qn12,qn13,qn18a,qn18b,qn18c,qn18d01,qn18dmnth,qn18dyear,qn18e,qn19b,qn20,qn24a,qn24b,qn25a,qn25b,qn25c,qn25d,qn26b,qn26d,qn26e,qn26estate,qn26f,qn26h,qn27a,qn27b01,qn27bmnth,qn27byear,qn27c,qn28a,qn28b,qn29b,qn29c,qn29c_months,cohort,ui_agect_arrival,qn30a,qn30d,qn31a,qn31d,qn31e,qn31f,qn31f_months,qn32a,qn32d,qn32e,qn33a,qn33d,qn33e,qn33f,qn33f_months,qn34a,qn34d,qn34e,qn34f,qn34f_months,qn35a,qn38a,qn38b,qn38c,ui_soi_pubassist,ui_soi,Weight_household,Weight_household_pop,personid,respondent,qn17_01,qn17_02,qn17_03,qn17_04,qn17_05,qn17_06,qn17_07,qn17_08,qn17_97,qn26ha_01,qn26ha_02,qn26ha_03,qn26ha_04,qn26ha_05,qn26ha_06,qn26ha_07,qn26ha_08,qn26ha_97,qn29a_01,qn29a_02,qn29a_03,qn29a_04,qn29a_05,qn29a_06,qn29a_07,qn29a_08,qn29a_09,qn29a_10,qn29a_11,qn29a_12,qn29a_97,qn29d_01,qn29d_02,qn29d_03,qn29d_04,qn29d_97,qn30b_01,qn30b_02,qn30b_03,qn30b_04,qn30b_05,qn31b_01,qn31b_02,qn31b_03,qn31b_04,qn31b_05,qn32b_01,qn32b_02,qn32b_03,qn32b_04,qn32b_05,qn33b_01,qn33b_02,qn33b_03,qn33b_04,qn33b_05,qn34b_01,qn34b_02,qn34b_03,qn34b_04,qn34b_05,ui_qn8a_annual,ui_qn10a_annual,ui_cashassist,ui_lfp,ui_emprate,ui_medicaidrma,ui_lpr,ui_school,ui_work,survey_year
1919,10001017.0,(record respondent name),3.0,self,now married (note: spouse need not live in hou...,70.0,male,iraq,iraq,chaldean,2013.0,west,NaN,8.0,primary,civil servant (civilian in local or national g...,"skilled tradesperson (carpenter, mechanic, plu...",not at all,not at all,yes,no,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,never worked in the u.s.,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,no,NaN,NaN,NaN,60.0,yes,NaN,NaN,reunification with relatives,yes,yes,refused,NaN,2014.0,NaN,no,no,health clinic,not covered in any month,NaN,2012 to 2013,55 or older,no,NaN,no,NaN,NaN,no months,NaN,no,NaN,NaN,yes,12.0,yes,every month,NaN,no,NaN,NaN,no months,NaN,no,rented for cash rent,1100.0,no,receives public assistance,"receives public assistance, but earnings missing",0.7017,67.9042,100010171.0,respondent,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,age,option not selected,option not selected,option not selected,option not selected,help with homework,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,option not selected,other insurance,option not selected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Respondent,household member #2,option not selected,option not selected,option not selected,NaN,NaN,NaN,NaN,NaN,NaN,NaN,receives cash assistance,not in labor force,not in labor force,individual does not receive rma/medicaid,already adjusted lpr status,none,not working now and never worked in past,2017
3244,10001469.0,(record respondent name),2.0,self,divorced,41.0,female,other,other,other,2013 or earlier,northeast,NaN,18.0,university degree (other than medical),employee in private sector,other,well,very well,yes,yes,no,yes,no,NaN,40.0,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,40.0,44000.0,(record month),april,2013 or earlier,yes,health care/education/social service,"employee of a private company, business, or in...",yes,1.0,yes,yes,bachelor's degree,yes,18.0,yes,NaN,NaN,reunification with relatives,yes,yes,(record month),june,2018 or later,NaN,no,no,no regular source,no - number of months not covered (range: 02-11),4.0,2013 to 2014,25 to 39 years,no,NaN,no,NaN,NaN,number of months,3.0,no,NaN,NaN,no,NaN,NaN,no months,NaN,no,NaN,NaN,don't know,NaN,no,rented for cash rent,1400.0,yes,receives public assistance,receives both,1.

In [35]:
# # There are repeats, but looking deeper, they don't actually seem like the same people
# c = rsp_df['personid'].value_counts()
# repeats = set()
# for i, v in c.items():
#     if v > 1:
#         repeats.add(i)

# rsp_df[rsp_df['personid'] == 100006711.0]

In [36]:
# Columns that shouldn't be any help given our granularity
misc_drop = [
    "hhid",
    "personid",
    "respondent",
    "qn1a",
    "qn3b",
    "qn26e",
    "qn1l",
    "qn5c",
    "qn8b",
    "qn10b",
    "qn1b"
]

# Binary answers where the _XX refers to if household member _XX was recipient
# Ex. qn31_03 is just wether household member 3 received food stamps
non_rsp_drop = [
    "qn30b_01",
    "qn30b_02",
    "qn30b_03",
    "qn30b_04",
    "qn30b_05",
    "qn31b_01",
    "qn31b_02",
    "qn31b_03",
    "qn31b_04",
    "qn31b_05",
    "qn32b_01",
    "qn32b_02",
    "qn32b_03",
    "qn32b_04",
    "qn32b_05",
    "qn33b_01",
    "qn33b_02",
    "qn33b_03",
    "qn33b_04",
    "qn33b_05",
    "qn34b_01",
    "qn34b_02",
    "qn34b_03",
    "qn34b_04",
    "qn34b_05",
]

rsp_df.drop(misc_drop, axis=1, inplace=True)
rsp_df.drop(non_rsp_drop, axis=1, inplace=True)

# rsp_df.drop(rsp_df[rsp_df['qn6b'] == "don't know"].index)

null_counts = rsp_df.isnull().sum()
print(null_counts[null_counts > 1000])

qn4j               3133
qn5b               1436
qn6a               1436
qn6b               4293
qn7                1436
qn8a               4168
qn9                4293
qn10a              4479
qn11a              3100
qn11aa             3884
qn12               3878
qn13               3100
qn18dmnth          1505
qn18dyear          1016
qn24b              3849
qn25b              3913
qn25c              4051
qn25d              4051
qn26estate         4358
qn27bmnth          2613
qn27byear          1448
qn27c              3097
qn29c_months       4168
qn30d              1993
qn31d              4259
qn31e              4259
qn31f_months       3334
qn32d              4390
qn32e              4390
qn33d              3731
qn33e              3731
qn33f_months       3941
qn34d              4444
qn34e              4444
qn34f_months       3736
qn17_01            3440
qn17_02            3440
qn17_03            3440
qn17_04            3440
qn17_05            3440
qn17_06            3440
qn17_07         

In [37]:
to_drop = []

# Answer entry changed by year
rsp_df["qn1c"].replace("now married (note: spouse need not live in household)", "now married", inplace=True)

rsp_df["qn1h"] = rsp_df.apply(
    lambda row: row["qn1g"] if row["qn1h"] in ['other','none','united states'] else row["qn1h"], axis=1
)

rsp_df["qn1k"].replace([98.0,99.0, 'north east'], ["don't know","refused", 'northeast'], inplace=True)

rsp_df["qn4j"] = rsp_df.apply(
    lambda row: row["qn4e"] if pd.isna(row["qn4j"]) else row["qn4j"], axis=1
)

# Question is if they worked more than 1 job in last week, blanks indicate they haven't worked any jobs in last week
rsp_df["qn5b"].fillna(value="no", inplace=True)

# Need this column to be numeric for separate calculation
rsp_df["qn6a"].fillna(value=0.0, inplace=True)
rsp_df["qn6a"].replace("don't know", "", inplace=True)
rsp_df["qn6a"].replace("refused", "", inplace=True)
rsp_df["qn6a"] = pd.to_numeric(rsp_df["qn6a"])

# Empties indicate they selected they didn't work second job, question is how many hours worked at second job.
# rsp_df['qn6b'] = rsp_df['qn6b'].cat.add_categories('0.0')
rsp_df["qn6b"].fillna(value=0.0, inplace=True)
rsp_df["qn6b"].replace("don't know", "", inplace=True)
rsp_df["qn6b"].replace("refused", "", inplace=True)
rsp_df["qn6b"] = pd.to_numeric(rsp_df["qn6b"])

# Need this column to be numeric for separate calculation
# Not sure how to fill empties. Adding 0 would indicate they worked for free which and could throw off calcs
rsp_df["qn7"].replace("don't know", "", inplace=True)
rsp_df["qn7"].replace("refused", "", inplace=True)
rsp_df["qn7"] = pd.to_numeric(rsp_df["qn7"])

# Empties exist if subject knew how many hours worked at primary job
rsp_df["qn8a"] = rsp_df.apply(
    lambda row: row["qn6a"] * row["qn7"] if pd.isna(row["qn8a"]) else row["qn8a"],
    axis=1,
)

# Question is on hours worked at second job in last week. Empties are if they indicated they didn't work second job.
# rsp_df['qn9'] = rsp_df['qn9'].cat.add_categories('0.0')
# rsp_df['qn9'].fillna(value='0.0', inplace = True)
rsp_df["qn9"].fillna(value=0.0, inplace=True)
rsp_df["qn9"].replace("don't know", "", inplace=True)
rsp_df["qn9"].replace("refused", "", inplace=True)
rsp_df["qn9"] = pd.to_numeric(rsp_df["qn9"])


# Empties exist if subject knew how many hours worked at primary job
rsp_df["qn10a"] = rsp_df.apply(
    lambda row: row["qn6b"] * row["qn9"] if pd.isna(row["qn10a"]) else row["qn10a"],
    axis=1,
)

# Question is skipped if indicated they worked in past week. This fill imputes the fields if the question had been asked.
rsp_df["qn11a"].fillna(value="yes", inplace=True)

# Only empty if they had worked in the past week. Imputing 0's for if the question had been asked.
rsp_df["qn11aa"].fillna(value=0, inplace=True)

# Changing def from "were you not working due to temp absence or layoff from job" to "Were you on temp absence or layoff"
rsp_df["qn12"].fillna(value="no, was not temporarily absent or on layoff", inplace=True)

# Question is if they've looked for work in past 4 weeks. Filling empties with 'employed' since they worked in past week
rsp_df["qn13"] = rsp_df["qn13"].astype("category")
rsp_df["qn13"] = rsp_df["qn13"].cat.add_categories("employed")
rsp_df["qn13"].fillna(value="employed", inplace=True)

# Ask how many weeks worked, blank indicates they haven't
rsp_df["qn18a"].fillna(value=0.0, inplace=True)
rsp_df["qn18a"].replace("don't know", "", inplace=True)
rsp_df["qn18a"].replace("refused", "", inplace=True)
rsp_df["qn18a"] = pd.to_numeric(rsp_df["qn18a"])

# How many hours usually worked, blank indicates they haven't
rsp_df["qn18b"].fillna(value=0.0, inplace=True)
rsp_df["qn18b"].replace("don't know", "", inplace=True)
rsp_df["qn18b"].replace("refused", "", inplace=True)
rsp_df["qn18b"] = pd.to_numeric(rsp_df["qn18b"])

# Income before taxes, blank indicates they haven't worked
rsp_df["qn18c"].fillna(value=0.0, inplace=True)
rsp_df["qn18c"].replace("don't know", "", inplace=True)
rsp_df["qn18c"].replace("refused", "", inplace=True)
rsp_df["qn18c"] = pd.to_numeric(rsp_df["qn18c"])

# Adding to Drop as it doesn't contain any information
to_drop.append("qn18d01")

# # Date of first job, commented out for now
# rsp_df['qn18dyear'].replace("2013 or earlier", 2013.0, inplace=True)
# rsp_df['qn18dyear'].replace("2018 or later", 2018.0, inplace=True)
# rsp_df['qn18d'] = rsp_df["qn18dmnth"].astype(str) + " " + rsp_df['qn18dyear'].astype(str)

# Blank if they are not working
rsp_df["qn20"].fillna(value="none/not working", inplace=True)

# Empties indicate they had not received any job training in past 12 months i.e. they have "0" training hours
rsp_df["qn24b"].fillna(value=0, inplace=True)

# Question is if they were attending school to obtain degree, blank indicates they are not in school
rsp_df["qn25b"].fillna(value="not in school", inplace=True)

# Question is if they were attending school to obtain degree, blank indicates they are not in school
rsp_df["qn25c"].fillna(value="not pursuing degree", inplace=True)

# Question is if they received degree blank indicates they are not in school
# Slight concern on this one as it might indicate they don't have a degree
rsp_df["qn25d"].fillna(value="not in school", inplace=True)

# These questions aren't asked if subject is looking for job, filling the empties with NA.
# In General, we might be able to decode these into 1 column before editing empties if we want.
for xx in ["01", "02", "03", "04", "05", "06", "07", "08", "97"]:
    qn = "qn17_" + xx
    qn26 = "qn26ha_" + xx
    rsp_df[qn] = rsp_df[qn].astype("category")
    rsp_df[qn26] = rsp_df[qn26].astype("category")
    rsp_df[qn] = rsp_df[qn].cat.add_categories("NA")
    rsp_df[qn26] = rsp_df[qn26].cat.add_categories("NA")
    rsp_df[qn].fillna(value="NA", inplace=True)
    rsp_df[qn26].fillna(value="NA", inplace=True)
    if xx in ["01", "02", "03", "04", "97"]:
        qn29 = "qn29d_" + xx
        rsp_df[qn29] = rsp_df[qn29].astype("category")
        rsp_df[qn29] = rsp_df[qn29].cat.add_categories("NA")
        rsp_df[qn29].fillna(value="NA", inplace=True)

# They indicated they didn't live in currents state a year ago, not perfect, but using initial state could be effective
rsp_df["qn26estate"] = rsp_df.apply(
    lambda row: row["qn1k"] if pd.isna(row["qn26estate"]) else row["qn26estate"], axis=1
)

# They haven't applied for citizenship, filling with NA
rsp_df["qn27bmnth"].fillna(value="NA", inplace=True)
rsp_df["qn27byear"].fillna(value="NA", inplace=True)

# Question is on if subject plans to apply for citizenship, empties indicate they already are
rsp_df["qn27c"].fillna(value="yes", inplace=True)

# Replacing for consistency
rsp_df["qn29c"].replace(
    "no - number of months not covered (range: 02-11)",
    "no - number of months not covered",
    inplace=True,
)

# Imputing based on answer to previous question
rsp_df["qn29c_months"] = rsp_df.apply(
    lambda row: 12.0
    if (pd.isna(row["qn29c_months"]) and row["qn29c"] == "yes - covered in all months")
    else row["qn29c_months"],
    axis=1,
)
rsp_df["qn29c_months"] = rsp_df.apply(
    lambda row: 0.0
    if (pd.isna(row["qn29c_months"]) and row["qn29c"] == "not covered in any month")
    else row["qn29c_months"],
    axis=1,
)
rsp_df["qn29c_months"] = rsp_df.apply(
    lambda row: 1.0
    if (pd.isna(row["qn29c_months"]) and row["qn29c"] == "not covered 1 month or less")
    else row["qn29c_months"],
    axis=1,
)

# Question is about total months on food stamps, empties indicate they never were.
rsp_df["qn30d"].fillna(value=0.0, inplace=True)
rsp_df["qn30d"].replace("don't know", "", inplace=True)
rsp_df["qn30d"].replace("refused", "", inplace=True)
rsp_df["qn30d"].replace("less than one month", 0.0, inplace=True)
rsp_df["qn30d"] = pd.to_numeric(rsp_df["qn30d"])


# Question is about total months on TANF, empties indicate they never were.
rsp_df["qn31d"].fillna(value=0.0, inplace=True)
rsp_df["qn31d"].replace("don't know", "", inplace=True)
rsp_df["qn31d"].replace("refused", "", inplace=True)
rsp_df["qn31d"] = pd.to_numeric(rsp_df["qn31d"])

# Question is if they used TANF in last month, empties indicate they never were on TANF
rsp_df["qn31e"].fillna(value="no", inplace=True)

# Answers are empty if subject answered they had never received TANF, or have always received TANF
## Not sure what to fill in the case the subject answered 'every month' to if they receive TANF
rsp_df["qn31f_months"] = rsp_df.apply(
    lambda row: 0.0
    if (pd.isna(row["qn31f_months"]) and row["qn31f"] == "no months")
    else row["qn31f_months"],
    axis=1,
)

# Question is about total months on RCA, empties indicate they never were.
rsp_df["qn32d"].fillna(value=0.0, inplace=True)
rsp_df["qn32d"].replace("don't know", "", inplace=True)
rsp_df["qn32d"].replace("refused", "", inplace=True)
rsp_df["qn32d"] = pd.to_numeric(rsp_df["qn32d"])

# Question is if they used RCA in last month, empties indicate they never were on TANF
rsp_df["qn32e"].fillna(value="no", inplace=True)

# Question is about total months on SSI, empties indicate they never were.
rsp_df["qn33d"].fillna(value=0.0, inplace=True)
rsp_df["qn33d"].replace("don't know", "", inplace=True)
rsp_df["qn33d"].replace("refused", "", inplace=True)
rsp_df["qn33d"] = pd.to_numeric(rsp_df["qn33d"])

# Question is if they used SSI in last month, empties indicate they never were on TANF
rsp_df["qn33e"].fillna(value="no", inplace=True)

# Answers are empty if subject answered they had never received TANF, or have always received SSI
## Not sure what to fill in the case the subject answered 'every month' to if they receive SSI
rsp_df["qn33f_months"] = rsp_df.apply(
    lambda row: 0.0
    if (pd.isna(row["qn33f_months"]) and row["qn33f"] == "no months")
    else row["qn33f_months"],
    axis=1,
)

# Question is about total months on GA, empties indicate they never were.
rsp_df["qn34d"].fillna(value=0.0, inplace=True)
rsp_df["qn34d"].replace("less than one month", 0.5, inplace=True)
rsp_df["qn34d"].replace("don't know", "", inplace=True)
rsp_df["qn34d"].replace("refused", "", inplace=True)
rsp_df["qn34d"] = pd.to_numeric(rsp_df["qn34d"])

# Question is if they used GA in last month, empties indicate they never were on TANF
rsp_df["qn34e"].fillna(value="no", inplace=True)

# Answers are empty if subject answered they had never received TANF, or have always received TANF
## Not sure what to fill in the case the subject answered 'every month' to if they receive GA
rsp_df["qn34f_months"] = rsp_df.apply(
    lambda row: 0.0
    if (pd.isna(row["qn34f_months"]) and row["qn34f"] == "no months")
    else row["qn34f_months"],
    axis=1,
)

# Filling in some empties to align with fills for qn8a, if qn8a is empty, fill with 0
rsp_df["ui_qn8a_annual"] = rsp_df.apply(
    lambda row: row["qn8a"] * 52
    if pd.isna(row["ui_qn8a_annual"])
    else row["ui_qn8a_annual"],
    axis=1,
)
rsp_df["ui_qn8a_annual"].fillna(value=0.0, inplace=True)

# Filling in some empties to align with fills for qn10a
rsp_df["ui_qn10a_annual"] = rsp_df.apply(
    lambda row: row["qn10a"] * 52
    if pd.isna(row["ui_qn10a_annual"])
    else row["ui_qn10a_annual"],
    axis=1,
)

rsp_df.drop(to_drop, axis=1, inplace=True)

In [38]:
# qn7 - Not sure how to fill empties. Adding 0 would indicate they worked for free which and could throw off calcs
# qn8a - Is reliant on qn7, if we decide on that, qn8 will be better
# qn18c - empties are not from people who answered "Don't know" or 'refused'
# qn18dmnth - blank if they never worked, or don't know when they started
# qn18dyear - blank if they never worked, or don't know when they started
# qn18e - blank if they weren't working. Filling with "no" implies they were working, and it didn't disqualify them from CA
# qn31f_months - Blanks are from if they answered they received this assistance every month, don't know, or refused
# qn33f_months - Blanks are from if they answered they received this assistance every month, don't know, or refused
null_counts = rsp_df.isnull().sum()
print(null_counts[null_counts > 100])

qn6a             111
qn7             1803
qn8a            1492
qn18a            607
qn18b            218
qn18c           1673
qn18dmnth       1505
qn18dyear       1016
qn18e            785
qn19b            785
qn27b01          712
qn29c_months     219
qn30d            132
qn31f_months     797
qn33f_months     898
qn34f_months     722
ui_school        141
dtype: int64


In [39]:
# Attempting to fill in missing data for remaining columns

# Set wage to 0 if person hasn't worked a job recently
rsp_df["qn7"] = rsp_df.apply(
    lambda row: 0 if row["qn5a"] == "no" else row["qn7"],
    axis=1,
)

# Set earnings to 0 if person hasn't worked a job recently
rsp_df["qn8a"] = rsp_df.apply(
    lambda row: 0 if row["qn5a"] == "no" else row["qn8a"],
    axis=1,
)

# Set weeks worked to 0 if person hasn't worked a job in US
rsp_df["qn18a"] = rsp_df.apply(
    lambda row: 0
    if row["qn5a"] == "no" and row["qn11a"] == "never worked in the u.s."
    else row["qn18a"],
    axis=1,
)

# Set hours worked to 0 if person hasn't worked a job in US
rsp_df["qn18b"] = rsp_df.apply(
    lambda row: 0
    if row["qn5a"] == "no" and row["qn11a"] == "never worked in the u.s."
    else row["qn18b"],
    axis=1,
)

# Set earnings to 0 if person hasn't worked a job in US
rsp_df["qn18c"] = rsp_df.apply(
    lambda row: 0
    if row["qn5a"] == "no" and row["qn11a"] == "never worked in the u.s."
    else row["qn18c"],
    axis=1,
)

# Set year to "never worked" if person hasn't worked a job in US
rsp_df["qn18dyear"] = rsp_df.apply(
    lambda row: "never worked in the u.s."
    if row["qn5a"] == "no" and row["qn11a"] == "never worked in the u.s."
    else row["qn18dyear"],
    axis=1,
)

# Set income / cash assistance to "never worked" if person hasn't worked a job in US
rsp_df["qn18e"] = rsp_df.apply(
    lambda row: "never worked in the u.s."
    if row["qn5a"] == "no" and row["qn11a"] == "never worked in the u.s."
    else row["qn18e"],
    axis=1,
)

# Set industry to "never worked" if person hasn't worked a job in US
rsp_df["qn19b"] = rsp_df.apply(
    lambda row: "never worked in the u.s."
    if row["qn5a"] == "no" and row["qn11a"] == "never worked in the u.s."
    else row["qn19b"],
    axis=1,
)

In [40]:
# Removing columns

# Remove due to large amount of missing data that can't be logically filled
rsp_df.drop("qn18c", axis=1, inplace=True, errors="ignore")

# Drop 18a, 18d_month as level of granularity is not necessary
rsp_df.drop("qn18a", axis=1, inplace=True, errors="ignore")
rsp_df.drop("qn18dmnth", axis=1, inplace=True, errors="ignore")

# Removing qn27b01 (administrative column which is not needed)
rsp_df.drop("qn27b01", axis=1, inplace=True, errors="ignore")

# Removing qn31-33f (unnecessary granularity)
rsp_df.drop(
    ["qn31f_months", "qn33f_months", "qn34f_months"],
    axis=1,
    inplace=True,
    errors="ignore",
)

In [41]:
# Replaceing some values caught post imputation
rsp_df["qn1d"].replace(["75 or older","75+"], [75,75], inplace=True)
rsp_df["qn2a"].replace("20 or more", 20, inplace=True)
rsp_df["qn26b"].replace("less than 1 month", 0, inplace=True)
rsp_df["qn26estate"].replace([98.0,99.0], ["don't know","refused"], inplace=True)
rsp_df["cohort"].replace(["2014 to 2015","2012 to 2013","2013 to 2014","2015 to 2016","2016 to 2017"]\
                             , ["2014","2012","2013","2015","2016"], inplace=True)
rsp_df["qn1jyear"].replace(["2016 or later","2012 or earlier","2017 or later","2013 or earlier"]\
                             , ["2016","2012","2017","2013"], inplace=True)
rsp_df["qn18dyear"].replace(["never worked in the u.s.","2018 or later","2013 or earlier"]\
                             , ["","2018","2013"], inplace=True)
rsp_df["qn27byear"].replace(["2018 or later","2013 or earlier"]\
                             , ["2018","2013"], inplace=True)

In [42]:
null_counts = rsp_df.isnull().sum()
print(null_counts[null_counts > 10])

qn1jyear         74
qn6a            111
qn6b             13
qn7             369
qn8a             58
qn9              56
qn18b           218
qn18dyear       238
qn29c_months    219
qn30d           132
qn31d            18
qn32d            16
qn33d            29
qn34d            11
qn38b            93
ui_school       141
dtype: int64


In [43]:
# Drop data from non-respondents / errors
rsp_df.dropna(inplace=True)

len(rsp_df)

3282

In [44]:
rsp_df.replace(["don't know","refused","high school student"], ["","",""], inplace=True)

In [45]:
rsp_df.to_csv("../Data/processed/dataset_2016-19", index=False)